# UDIs, UMA

This notebook aims to discover a bit about the SIE API from Banxico. We will particularly focus (for now) on getting UDI data, fundamental for anyone working in finance in Mexico as most government insurance products are given in UDI. 

<u>Note:</u> For SIE, limits are 200 API calls in 5 minutes and 10,000 API calls per day.

### SIE Series ID

SIE gives an ID to each of its time series. A short list of the main time series we will use here is :
- <B>SP68257</B> : Time series of UDI/MXN.
- <B>SF43783</B> : TIIE a 28 dias.
- <B>SF61745</B> : Tasa objetivo of Banxico
- <B>SF331451</B> : Daily values (in %) of TIIE de Fondeo, the new MXN risk-free rate officially used in credit contracts as of 01/01/2025 as part of the post-LIBOR transition. 

### Setting up parameters

In [ ]:
#############################################
################# Tokens ####################
#############################################

# Get a token on page : 
sie_token = "MyToken"
# Get a token there : https://www.inegi.org.mx/app/desarrolladores/generatoken/Usuarios/token_Verify
inegi_token = "MyToken"

#############################################
############# API endpoints #################
#############################################

sie_endpoint = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"

uma_endpoint = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/539260/es/0700/true/BISE/2.0/{inegi_token}?type=json"


#############################################
############ SIE Series ID ##################
#############################################

SIE_series_IDs = {"UDI": "SP68257",
                  "TIIE_De_Fondeo":"SF331451",
                  "TIIE_28_dias":"SF43783"}

### Importing relevant libraries

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, interact

### Common functions

In [ ]:
def fetch_sie_series(token, endpoint, series_key):
    """
    Get data from SIE API for a given series ID.
    """
    headers = {"Bmx-Token": token}

    # Handle wrong series key
    try:
        series_id = SIE_series_IDs[series_key]
    except KeyError:
        print(f"Unvalid key. Try a key amongst {tuple(SIE_series_IDs.keys())}")
    request_url = f"{endpoint}/{series_id}/datos"
    # API request
    response = requests.get(request_url, headers=headers)

    # Check for errors
    if response.status_code != 200:
        print("Error response:", response.text)  # Print error details
        response.raise_for_status()

    # Parse response
    data = response.json()
    series_data = data["bmx"]["series"][0]["datos"]

    # Extract dates and values
    dates = [entry["fecha"] for entry in series_data]
    values = [float(entry["dato"]) for entry in series_data]

    # Create Pandas Series (specifying dayfirst=True)
    output = pd.Series(data=values, index=pd.to_datetime(dates, dayfirst=True), name=series_key)
    return output

In [ ]:
# Get UDI data
udi_series = fetch_sie_series(sie_token, sie_endpoint, "UDI")

## How can UDIs be used ?

- Seguro SOFIPO : 25,000 UDIs. You can find the list of SOFIPOs there : <url>https://www.cnbv.gob.mx/Entidades-Autorizadas/Paginas/Sofipos.aspx</url>
- Seguro IPAB : 400,000 UDIs

In [ ]:
# Get current time
today = datetime.now()

# Convert to the format YYYY-MM-DD
today_sie_format = today.strftime('%Y-%m-%d')

udi_today = udi_series.loc[today_sie_format]

# Displays today's amount for SOFIPO insurance in MXN
seguro_sofipo_today = 25000*udi_today
print(f"Today, on {today_sie_format}, the amount of the seguro SOFIPO is {format(seguro_sofipo_today, ',')} MXN \n")

# Displays today's amount for IPAB insurance in MXN
seguro_ipab_today = 400000*udi_today
print(f"Today, on {today_sie_format}, the amount of the seguro IPAB is {format(seguro_ipab_today, ',')} MXN")


### TIIE de fondeo

https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?sector=18&accion=consultarCuadroAnalitico&idCuadro=CA51&locale=es

In [ ]:
# Get TIIE de fondeo data

tiie_fondeo_series = fetch_sie_series(sie_token, sie_endpoint, "TIIE_De_Fondeo")

In [ ]:
# Create the Plotly chart
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=tiie_fondeo_series.index,
    y=tiie_fondeo_series.values,
    mode="lines",
    name="TIIE De Fondeo",
    line=dict(color="blue")
))
fig.update_layout(
    title="TIIE De Fondeo",
    xaxis_title="Date",
    yaxis_title="TIIE De Fondeo",
    template="plotly_white",
    width=900,
    height=500
)
fig.show()

## UMA

UMA comes from INEGI. Values can be found here : <url>https://www.inegi.org.mx/temas/uma/</url>

Scraping it is as simple as it gets, but we'll rather use the Banco de Indicadores API from INEGI.

UMAs are quite important when it comes to taxes in Mexico, for both individuals and companies.

In [ ]:
# Function to fetch UMA values
def fetch_uma_values(endpoint):
    # API Request
    response = requests.get(endpoint)
    response.raise_for_status()  # Raise an exception for HTTP errors
    
    # Parse the JSON response
    data = response.json()
    series = data["Series"]

    if not series or "OBSERVATIONS" not in series[0]:
        raise ValueError("No UMA data found in the API response.")
    
    # Extract observations
    observations = series[0]["OBSERVATIONS"]
    valid_data = [
        {"Year": int(obs["TIME_PERIOD"]), "Daily UMA": float(obs["OBS_VALUE"])}
        for obs in observations
        if obs["OBS_VALUE"] is not None  # Skip entries with no value
    ]
    
    # Convert to Pandas DataFrame
    uma_df = pd.DataFrame(valid_data)
    uma_df.set_index("Year", inplace=True)
    
    # Add columns for monthly and yearly UMA
    uma_df["Monthly UMA"] = np.round(uma_df["Daily UMA"] * 30.4,2)  # 30.4 days per month, must be rounded to 2 decimals
    uma_df["Yearly UMA"] = uma_df["Monthly UMA"] * 12   # 12 months per year
    
    return uma_df

# Fetch and display UMA data
try:
    uma_data = fetch_uma_values(uma_endpoint)
    print(uma_data)
except requests.RequestException as e:
    print(f"API request failed: {e}")
except ValueError as ve:
    print(f"Data processing error: {ve}")

#### Plan personal de Retiro, art xxx

The Plan Personal de Retiro is the Mexican IRA. You can invest your money through it, and will have to pay 20% taxes on money leaving your PPR, above a certain threshold.

This threshold is:
- 90 yearly UMAs if you choose a lump sum withdrawal.
- 15 UMAs yearly UMAs each year if you choose to only withdraw part of your money. (Be aware that not all institutions will allow you to make partial withdrawals. Some will just sell out your assets when you come to the retirement age and send you the lump sum.)

Also, the amount of money invested in a PPR during a fiscal year can be deduced from your taxable income, if it is inferior or equal to 10% of your income, and up to 5 yearly UMA, whichever is lowest.

In [ ]:
current_year = datetime.now().year

current_year_umas = uma_data.loc[current_year]

print(f"This year, you can invest in a PPR up to 10% of your taxable income, or {np.round(current_year_umas['Yearly UMA']*5,2)} MXN, whichever is the lowest. \n")

print(f"If you make partial withdrawals, you can withdraw up to {np.round(current_year_umas['Yearly UMA']*15,2)} MXN this year without paying taxes on it. \n")

print(f"If you withdraw all your fund from your PPR, you can withdraw up to {np.round(current_year_umas['Yearly UMA']*90,2)} MXN this year, without paying taxes on it.")